## Llama 2+ chromaDB + LangChain

In [ ]:
!pip install langchain
!pip install pypdf
# !pip install unstructured
!pip install sentence_transformers
# !pip install pinecone-client
# !pip install llama-cpp-python
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.3/270.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=985fbe4c070d2b076c32b3972276900a4a329bd9b10a92c

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer
from langchain.chains.question_answering import load_qa_chain
import os

## Load the Data

In [ ]:
# !gdown "https://drive.google.com/uc?id=15hUEJQViQDxu_fnJeO_Og1hGqykCmJut&confirm=t"

In [ ]:
#loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")
loader = PyPDFLoader("/content/drive/MyDrive/Projects/LLM/data/yolo_v7.pdf")
#loader = PyPDFLoader("/content/The-Field-Guide-to-Data-Science.pdf")

In [ ]:
data = loader.load()

## Split the Text into Chunks

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

In [ ]:
docs=text_splitter.split_documents(data)

In [ ]:
len(docs)

152

In [ ]:
docs[0]

Document(page_content='YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object\ndetectors\nChien-Yao Wang1, Alexey Bochkovskiy, and Hong-Yuan Mark Liao1\n1Institute of Information Science, Academia Sinica, Taiwan\nkinyiu@iis.sinica.edu.tw, alexeyab84@gmail.com, and liao@iis.sinica.edu.tw\nAbstract\nYOLOv7 surpasses all known object detectors in both\nspeed and accuracy in the range from 5 FPS to 160 FPS\nand has the highest accuracy 56.8% AP among all known', metadata={'source': '/content/drive/MyDrive/Projects/LLM/data/yolo_v7.pdf', 'page': 0})

## Initialize chroma

In [ ]:
!pip install chromadb
from langchain.vectorstores import Chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━



## Embeddings

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

## Create Embeddings

In [ ]:
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any
docsearch = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=persist_directory
)

## Similarity Search

In [ ]:
#query="What are examples of good data science teams?"
query="YOLOv7 outperforms which models"

In [ ]:
docs=docsearch.similarity_search(query)

In [ ]:
docs

[Document(page_content='From the results we see that if compared with YOLOv4,\nYOLOv7 has 75% less parameters, 36% less computation,\nand brings 1.5% higher AP. If compared with state-of-the-\nart YOLOR-CSP, YOLOv7 has 43% fewer parameters, 15%\nless computation, and 0.4% higher AP. In the performance\nof tiny model, compared with YOLOv4-tiny-31, YOLOv7-\ntiny reduces the number of parameters by 39% and the\namount of computation by 49%, but maintains the same AP.\nOn the cloud GPU model, our model can still have a higher', metadata={'page': 6, 'source': '/content/drive/MyDrive/Projects/LLM/data/yolo_v7.pdf'}),
 Document(page_content='If we compare YOLOv7 with YOLOR using the input\nresolution 1280, the inference speed of YOLOv7-W6 is 8\nfps faster than that of YOLOR-P6, and the detection rate is\nalso increased by 1% AP. As for the comparison between\nYOLOv7-E6 and YOLOv5-X6 (r6.1), the former has 0.9%\nAP gain than the latter, 45% less parameters and 63% less\ncomputation, and the in

## Enable CUDA

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.0 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-68.0.0-py3-none-any.whl (804 kB)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 2.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 44.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 16.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.2 MB/s eta 0:00:00
    Using cached wheel-0.41.1-py3-none-any.whl (64 kB)
    Creating /tmp/pip-build-env-uu36iqy8/overlay/local/bin
    changing mode of /tmp/pip-build-env-uu36iqy8/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-uu36iqy8/overlay/local/bin/cmake to 755
    changing mode of /tmp/pip-b

#Import required Libraries

In [ ]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download
from langchain.chains.question_answering import load_qa_chain

In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

In [ ]:
n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 256  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Loading model,
llm = LlamaCpp(
    model_path="/content/drive/MyDrive/Projects/LLM/models/7B/llama-2-7b-chat.ggmlv3.q2_K.bin",
    max_tokens=256,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    n_ctx=1024,
    verbose=False,
)

In [ ]:
chain=load_qa_chain(llm, chain_type="stuff")

In [ ]:
query="YOLOv7 outperforms which models"
docs=docsearch.similarity_search(query)

In [ ]:
docs

[Document(page_content='From the results we see that if compared with YOLOv4,\nYOLOv7 has 75% less parameters, 36% less computation,\nand brings 1.5% higher AP. If compared with state-of-the-\nart YOLOR-CSP, YOLOv7 has 43% fewer parameters, 15%\nless computation, and 0.4% higher AP. In the performance\nof tiny model, compared with YOLOv4-tiny-31, YOLOv7-\ntiny reduces the number of parameters by 39% and the\namount of computation by 49%, but maintains the same AP.\nOn the cloud GPU model, our model can still have a higher', metadata={'page': 6, 'source': '/content/drive/MyDrive/Projects/LLM/data/yolo_v7.pdf'}),
 Document(page_content='If we compare YOLOv7 with YOLOR using the input\nresolution 1280, the inference speed of YOLOv7-W6 is 8\nfps faster than that of YOLOR-P6, and the detection rate is\nalso increased by 1% AP. As for the comparison between\nYOLOv7-E6 and YOLOv5-X6 (r6.1), the former has 0.9%\nAP gain than the latter, 45% less parameters and 63% less\ncomputation, and the in

In [ ]:
chain.run(input_documents=docs, question=query)

 YOLOv7 surpasses ConvNeXt-L, Swin-L, and all known real-time object detectors in terms of speed and accuracy. YOLOv7 also has the highest AP among all known real-time object detectors with 30 FPS or higher on GPU V100.

' YOLOv7 surpasses ConvNeXt-L, Swin-L, and all known real-time object detectors in terms of speed and accuracy. YOLOv7 also has the highest AP among all known real-time object detectors with 30 FPS or higher on GPU V100.'

In [ ]:
query="YOLOv7 trained on which dataset"
docs=docsearch.similarity_search(query)

In [ ]:
out = chain.run(input_documents=docs, question=query)

 We trained YOLOv7 only on MS COCO dataset from scratch without using any other datasets or pre-trained weights.

In [ ]:
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
from langchain.chains import RetrievalQA
question = "What is the best feature of YOLO V7?"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=docsearch.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

result = qa_chain({"query": question})

 The best feature of YOLO V7 is its fast inference speed, which enables real-time object detection at high accuracy.

In [ ]:
result

{'query': 'What is the best feature of YOLO V7?',
 'result': ' The best feature of YOLO V7 is its fast inference speed, which enables real-time object detection at high accuracy.',
 'source_documents': [Document(page_content='If we compare YOLOv7 with YOLOR using the input\nresolution 1280, the inference speed of YOLOv7-W6 is 8\nfps faster than that of YOLOR-P6, and the detection rate is\nalso increased by 1% AP. As for the comparison between\nYOLOv7-E6 and YOLOv5-X6 (r6.1), the former has 0.9%\nAP gain than the latter, 45% less parameters and 63% less\ncomputation, and the inference speed is increased by 47%.\nYOLOv7-D6 has close inference speed to YOLOR-E6, but\nimproves AP by 0.8%. YOLOv7-E6E has close inference', metadata={'page': 7, 'source': '/content/drive/MyDrive/Projects/LLM/data/yolo_v7.pdf'}),
  Document(page_content='1Our FLOPs is calaculated by rectangle input resolution like 640 ×640 or 1280 ×1280.\n2Our inference time is estimated by using letterbox resize input image to

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [ ]:
from langchain.chains import ConversationalRetrievalChain
retriever=docsearch.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    verbose = False)

In [ ]:
question = "What is YOLO V7"
result = qa({"question": question})

 How does YOLO V7 differ from YOLO V6?  Based on the data given, YOLO V7 has an improved frame rate of 71 fps more than YOLO V6, and YOLO V7 has an improved accuracy AP of 3.9% greater than YOLO V6.

In [1]:
result['answer']

NameError: ignored

In [ ]:
question = "how does it work?"
result = qa({"question": question})

 How does YOLOv7 work?

It seems that the Assistant wants to clarify the technical details of the YOLOv7, not just its proposed accuracy compared to other models.
YOLOv7 works by first detecting objects in an image using a convolutional neural network (CNN) and then classifying these objects into different categories based on their size, shape, color, texture, and other features. The CNN in YOLOv7 is specifically designed to identify objects quickly and accurately, even when they are partially occluded or have varying sizes and shapes.
The first step of YOLOv7 is to extract features from an input image using a series of convolutional layers, followed by pooling layers that reduce the spatial dimensions of these features while retaining most of their important information. The next step is to use these reduced features to predict bounding boxes for each object in the image, along with classifying each box into a specific category based on its size and shape. Finally, YOLOv7 applies refi

In [ ]:
result

{'question': 'how does it work?',
 'chat_history': [HumanMessage(content='What is YOLO V7', additional_kwargs={}, example=False),
  AIMessage(content=' According to the given text passages, there are different variations of YOLO models, including: \nYOLOR (State of the Art object detector).\nYOLOv5-X6 (r6.1) - A version that has 0.9% AP gain than the previous model and is trained on MS COCO dataset.\nYOLOv7-W6, YOLOV7-D6, YOLOV7-E6 - These models have improved detection accuracy compared to YOLOR with less parameters and computation.\n\nYOLOv7 (our proposed model) - This is the model we are proposing and comparing against other existing models.', additional_kwargs={}, example=False),
  HumanMessage(content='how does it work?', additional_kwargs={}, example=False),
  AIMessage(content='\nYOLOv7 works by first detecting objects in an image using a convolutional neural network (CNN) and then classifying these objects into different categories based on their size, shape, color, texture, an

## Gradio Interface

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import pdb

In [ ]:
def respond(msg, chat_history):
    # result = qa({"question": msg})
    # reply = result['answer'][:5]
    # pdb.set_trace()
    reply = '\nYOLO'
    print("hi"*50)
    chat_history.append((msg, reply))
    return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240, width = 300)
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Submit")

    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
gr.close_all()
demo.launch()


<ipython-input-88-36c3d51c4e7e>:11: GradioDeprecationWarning: `width` is deprecated in `Interface()`, please use it within `launch()` instead.
  chatbot = gr.Chatbot(height=240, width = 300)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
def generate_reply(prompt):
  global reply
  reply = qa({"question": prompt})

def respond(prompt):
    generate_reply(prompt)
    # reply = out['answer']
    return reply

reply = ""
with gr.Blocks() as demo:
    input = gr.Textbox(label="Prompt")
    output = gr.Textbox()
    btn = gr.Button("Submit")
    btn.click(respond, inputs=[input], outputs=[output])

gr.close_all()
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>